In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [2]:
import os
print(os.getcwd())

/Users/YiHsien/Documents/kaggle/AnimeRecommendations


In [3]:
anime = pd.read_csv('anime.csv')
rating = pd.read_csv('rating.csv')

In [4]:
mergedata = pd.merge(rating,anime,on=['anime_id','anime_id'])
mergedata = mergedata[mergedata['type']=='Movie']
mergedata.head(10)

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
98741,1,442,-1,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98742,5,442,4,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98743,54,442,-1,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98744,99,442,2,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98745,128,442,7,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98746,129,442,8,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98747,171,442,10,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98748,188,442,-1,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98749,190,442,5,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98750,201,442,10,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571


In [5]:
## train and test

In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(mergedata, test_size=0.3)
print(train.shape,test.shape)

(736398, 9) (315600, 9)


In [7]:
## 將電影類別換成數字

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin
class CounterEncoder(BaseEstimator, TransformerMixin):
    """依照出現頻率進行編碼, 頻率由高到低的index = 0, 1, 2, 3 ..., 以此類推"""
    def fit(self, y):
        counter = pd.Series(y).value_counts()
        self.enc = dict(zip([None] + counter.index.tolist(), range(len(counter) + 1)))
        self.invEnc = dict(zip(self.enc.values(), self.enc.keys()))
        self.classes_ = counter.index.values
        return self

    def transform(self, y):
        return pd.Series(y).map(self.enc).fillna(0).values

    def fit_transform(self, y, **fit_params):
        return self.fit(y).transform(y)
    
    def inverse_transform(self, y):
        return pd.Series(y).map(self.invEnc).values


class OrderedMapper(CounterEncoder):
    def fit(self, y):
        uniq = pd.Series(y).unique()
        self.enc = dict(zip(uniq, range(len(uniq))))
        self.invEnc = dict(zip(range(len(uniq)), uniq))
        self.classes_ = uniq
        return self

In [9]:
from collections import defaultdict, Counter, OrderedDict
def doMovies(movies):
    """處理 movie: genres 轉換成數字"""
    movies = movies.reset_index(drop=True)
    movies.loc[movies.genre == "(no genres listed)", "genre"] = ""
    movies['genre']=movies['genre'].fillna('')
    movies["genre"] = movies.genre.str.split(",")
    genreMap = Counter()
    movies.genre.map(genreMap.update)
    om = OrderedMapper().fit([e[0] for e in genreMap.most_common()])
    movies["genre"] = movies.genre.map(lambda lst: [om.enc[e] for e in lst])
    return movies, om

In [10]:
movie_trans, genres_enc = doMovies(anime)

In [26]:
n_genres = len(genres_enc.enc)

In [11]:
from sklearn.preprocessing import minmax_scale
movie_trans["members"] = minmax_scale(movie_trans.members.fillna(movie_trans.members.median()))
movie_trans.head()

/Users/YiHsien/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[13, 7, 9, 12]",Movie,1,9.37,0.197872
1,5114,Fullmetal Alchemist: Brotherhood,"[1, 15, 11, 3, 17, 27, 4]",TV,64,9.26,0.782770
2,28977,Gintama°,"[1, 6, 18, 30, 41, 2, 4]",TV,51,9.25,0.112689
3,9253,Steins;Gate,"[49, 47]",TV,24,9.17,0.664325
4,9969,Gintama&#039;,"[1, 6, 18, 30, 41, 2, 4]",TV,51,9.16,0.149186


In [12]:
## leave on out

In [13]:
#拿掉無評分的資料
train = train[train['rating_x']!=-1]
test = test[test['rating_x']!=-1]
print(train.shape,test.shape)

(589529, 9) (252624, 9)


In [35]:
u = pd.DataFrame()
u['user_id'] = mergedata['user_id'].unique()
u['user'] = range(0,len(mergedata['user_id'].unique()))
m = pd.DataFrame()
m['anime_id'] = mergedata['anime_id'].unique()
m['movie'] = range(0,len(mergedata['anime_id'].unique()))

In [36]:
train = pd.merge(train,u,on=['user_id'])
train = pd.merge(train,m,on=['anime_id'])
test = pd.merge(test,u,on=['user_id'])
test = pd.merge(test,m,on=['anime_id'])
train.drop(['user_id','anime_id'], axis=1, inplace=True)
test.drop(['user_id','anime_id'], axis=1, inplace=True)
train.rename(columns={'user':'user_id','movie':'anime_id'}, inplace=True)
test.rename(columns={'user':'user_id','movie':'anime_id'}, inplace=True)

In [37]:
movie_trans = pd.merge(movie_trans,m,on=['anime_id'])
movie_trans.drop(['anime_id'], axis=1, inplace=True)
movie_trans.rename(columns={'movie':'anime_id'}, inplace=True)

In [14]:
def loo_preprocess(data, movie_trans, train_hist=None, is_train=True):
    """以leave one out方式產生 train data, test data"""
    queue = []
    data = pd.merge(data[['user_id','anime_id','rating_x']],movie_trans[['anime_id','genre','rating','members']], how="left", on="anime_id")
    columns=["user_id", "anime_id", "genre", "rating","members", "candidate_movie_id","rating_x"]
    for u, df in data.groupby("user_id"):
        df = df.sort_values("rating_x", ascending=False)
        if not is_train:
            user_movies_hist = train_hist.query("user_id == {}".format(u)).anime_id
        for i, (_, r) in enumerate(df.iterrows()):
            if is_train:
                queue.append([int(r.user_id),
                              df.anime_id[:i].tolist() + df.anime_id[i + 1:].tolist(),
                              r.genre, r.rating, r.members, int(r.anime_id), r.rating_x])
            else:
                # queue.append([int(r.userId), df.movieId[:i].tolist() + df.movieId[i + 1:].tolist(), r.genres, r.avg_rating, r.year, int(r.movieId), r.rating])
                # all_hist = set(user_movies_hist.tolist() + df.movieId[:i].tolist())
                all_hist = set(user_movies_hist.tolist())
                queue.append([int(r.user_id),
                              list(all_hist - set([int(r.anime_id)])),
                              r.genre, r.rating, r.members, int(r.anime_id), r.rating_x])
    return pd.DataFrame(queue, columns=columns)

In [38]:
trProcessed = loo_preprocess(train, movie_trans)
teProcessed = loo_preprocess(test, movie_trans)

In [39]:
trProcessed.head()

,user_id,anime_id,genre,rating,members,candidate_movie_id,rating_x
0,1,"[5, 35, 28, 53, 37, 24, 34, 11, 36, 46, 45, 33...","[1, 15, 18, 41]",8.43,0.123776,38,9
1,1,"[38, 35, 28, 53, 37, 24, 34, 11, 36, 46, 45, 3...","[5, 11, 12]",8.93,0.459852,5,8
2,1,"[38, 5, 28, 53, 37, 24, 34, 11, 36, 46, 45, 33...","[5, 11, 7, 2]",8.44,0.285233,35,8
3,1,"[38, 5, 35, 53, 37, 24, 34, 11, 36, 46, 45, 33...","[1, 15, 6, 11, 3, 8]",7.66,0.098711,28,7
4,1,"[38, 5, 35, 28, 37, 24, 34, 11, 36, 46, 45, 33...","[13, 10, 12]",8.19,0.093929,53,7


In [40]:
teProcessed.head()

,user_id,anime_id,genre,rating,members,candidate_movie_id,rating_x
0,1,"[50, 55, 26, 40, 52, 42, 41, 31, 47, 1, 57, 39...","[1, 6, 18, 30, 41, 2, 4]",9.10,0.071534,54,10
1,1,"[54, 55, 26, 40, 52, 42, 41, 31, 47, 1, 57, 39...","[49, 47]",8.61,0.189779,50,9
2,1,"[54, 50, 26, 40, 52, 42, 41, 31, 47, 1, 57, 39...","[0, 21, 10]",7.97,0.030386,55,8
3,1,"[54, 50, 55, 40, 52, 42, 41, 31, 47, 1, 57, 39...","[1, 15, 32, 27, 2, 12]",8.15,0.212930,26,8
4,1,"[54, 50, 55, 26, 52, 42, 41, 31, 47, 1, 57, 39...","[5, 6, 11, 25, 9, 21]",7.87,0.023940,40,8


In [41]:
import tensorflow as tf
def do_multi(df, multi_cols):
    """對於multivalent的欄位, 需要增加一個column去描述該欄位的長度"""
    pad = tf.keras.preprocessing.sequence.pad_sequences
    ret = OrderedDict()
    for colname, col in df.iteritems():
        if colname in multi_cols:
            lens = col.map(len)
            ret[colname] = list(pad(col, padding="post", maxlen=lens.max()))
            ret[colname + "_len"] = lens.values
        else:
            ret[colname] = col.values
    return ret
def get_minibatches_idx(n, batch_size, shuffle=False):
    idx_list = np.arange(n, dtype="int32")
    if shuffle:
        np.random.shuffle(idx_list)
    minibatches = []
    minibatch_start = 0
    for i in range(n // batch_size):
        minibatches.append(idx_list[minibatch_start : minibatch_start + batch_size])
        minibatch_start += batch_size

    if (minibatch_start != n):
        # Make a minibatch out of what is left
        minibatches.append(idx_list[minibatch_start:])
    return minibatches
def dataFn(data, n_batch=128, shuffle=False):
    def fn():
        dataInner = data.copy()
        indices = get_minibatches_idx(len(dataInner), n_batch, shuffle=shuffle)
        for ind in indices:
            yield do_multi(dataInner.iloc[ind], ["anime_id", "genre"])
    return fn

for i, e in enumerate(dataFn(trProcessed, n_batch=5, shuffle=True)(), 1):
    # print(e)
    break
pd.DataFrame(e)


,user_id,anime_id,anime_id_len,genre,genre_len,rating,members,candidate_movie_id,rating_x
0,9260,"[210, 259, 196, 3, 216, 258, 159, 190, 221, 49...",40,"[5, 6, 26, 38, 4, 0]",5,8.21,0.021485,279,9
1,1319,"[134, 60, 165, 19, 135, 390, 63, 536, 26, 50, ...",109,"[5, 6, 11, 3, 4, 12]",6,6.99,0.095968,1,6
2,8434,"[60, 70, 5, 151, 207, 83, 173, 123, 295, 97, 6...",88,"[28, 9, 10, 0, 0, 0]",3,8.18,0.037590,181,8
3,44350,"[79, 155, 157, 112, 83, 50, 338, 63, 153, 81, ...",14,"[1, 3, 8, 2, 0, 0]",4,7.61,0.059914,41,9
4,17367,"[97, 113, 83, 460, 670, 329, 436, 60, 106, 82,...",30,"[0, 26, 7, 9, 2, 12]",6,8.81,0.236995,153,9


In [17]:
class ModelMfDNN(object):
    def __init__(self, n_items, n_genre, dim=32, learning_rate=0.01, reg=0.05, modelDir="./model"):
        self.n_items = n_items
        self.n_genre = n_genre
        self.dim = dim
        self.learning_rate = learning_rate
        self.reg = reg
        self.modelDir = modelDir
        graph = tf.Graph()
        with graph.as_default():
            # inputs parts
            graph_inputs(self)
            # embedding parts
            graph_embedding(self)
            # deep neural network
            graph_dnn(self)
            # computation parts
            graph_computation(self)
            # loss parts
            graph_loss(self)
            self.saver = tf.train.Saver(tf.global_variables())
            self.graph = graph
        
    def fit(self, sess, trainGen, testGen, reset=False, nEpoch=50):
        return fit(self, sess, trainGen, testGen, reset=reset, nEpoch=nEpoch)
    
    def predict(self, sess, user_queries, items):
        self.ckpt(sess, self.modelDir)
        return sess.run(self.pred, feed_dict={
            self.isTrain: False,
            self.user_id: user_queries["user_id"],
            self.anime_id: user_queries["anime_id"],
            self.anime_id_len: user_queries["anime_id_len"],

            self.genre: items["genre"],
            self.genre_len: items["genre_len"],
            self.rating: items["rating"],
            self.members: items["members"],
            self.candidate_movie_id: items["candidate_movie_id"]
        })
    
    def resetModel(self, modelDir):
        """刪除model dir"""
        shutil.rmtree(path=modelDir, ignore_errors=True)
        os.makedirs(modelDir)
        
    def ckpt(self, sess, modelDir):
        """load latest saved model"""
        latestCkpt = tf.train.latest_checkpoint(modelDir)
        if latestCkpt:
            self.saver.restore(sess, latestCkpt)
        return latestCkpt
    
    def feed_dict(self, data, mode="train"):
        ret = {
            self.user_id: data["user_id"],
            self.anime_id: data["anime_id"],
            self.anime_id_len: data["anime_id_len"],
            self.genre: data["genre"],
            self.genre_len: data["genre_len"],
            self.rating: data["rating"],
            self.members: data["members"],
            self.candidate_movie_id: data["candidate_movie_id"]
        }
        ret[self.isTrain] = False
        if mode != "infer":
            ret[self.rating] = data["rating"]
            if mode == "train":
                ret[self.isTrain] = True
            elif mode == "eval":
                pass
        return ret

    def epochLoss(self, sess, dataGen, tpe="rmse"):
        totLoss, totCnt = 0, 0
        for data in dataGen():
            lossTensor = self.rmse_loss if tpe == "rmse" else self.mae_loss
            loss = sess.run(lossTensor, feed_dict=self.feed_dict(data, mode="eval"))
            totLoss += (loss ** 2 if tpe == "rmse" else loss) * len(data["anime_id"])
            totCnt += len(data["anime_id"])
        return np.sqrt(totLoss / totCnt) if tpe == "rmse" else totLoss / totCnt
    
    def evaluateRMSE(self, sess, dataGen):
        """計算root mean square error"""
        self.ckpt(sess, self.modelDir)
        return self.epochLoss(sess, dataGen, tpe="rmse")

    def evaluateMAE(self, sess, dataGen):
        """計算 mean absolutely error"""
        self.ckpt(sess, self.modelDir)
        return self.epochLoss(sess, dataGen, tpe="mae")

In [29]:
def graph_inputs(self):
    # inputs/id_user:0
    with tf.variable_scope("inputs"):
        self.isTrain = tf.placeholder(tf.bool, None)
        # user data
        self.user_id = tf.placeholder(tf.int32, [None])
        self.anime_id = tf.placeholder(tf.int32, [None, None])
        self.anime_id_len = tf.placeholder(tf.int32, [None])
        # item data
        self.genre = tf.placeholder(tf.int32, [None, None])
        self.genre_len = tf.placeholder(tf.int32, [None])
        self.rating = tf.placeholder(tf.float32, [None])
        self.member = tf.placeholder(tf.float32, [None])
        self.candidate_movie_id = tf.placeholder(tf.int32, [None])
        self.rating_x = tf.placeholder(tf.float32, [None])

In [31]:
def graph_embedding(self):
    init_fn = tf.glorot_normal_initializer()
    emb_init_fn = tf.glorot_uniform_initializer()
    self.b_global = tf.Variable(emb_init_fn(shape=[]), name="b_global")
    with tf.variable_scope("embedding"):
        self.w_anime_id = tf.Variable(emb_init_fn(shape=[self.n_items, dim]), name="w_anime_id")
        self.b_anime_id = tf.Variable(emb_init_fn(shape=[dim]), name="b_anime_id")
        self.w_candidate_movie_id = tf.Variable(init_fn(shape=[self.n_items, dim]), name="w_candidate_movie_id")
        self.b_candidate_movie_id = tf.Variable(init_fn(shape=[dim + 8 + 2]), name="b_candidate_movie_id")
        self.w_genre = tf.Variable(emb_init_fn(shape=[self.n_genre, 8]), name="w_genre")

        # query_movie embedding
        '''sqrtn aggregation(pooling), X: data, W: weight
               X_1*W_1 + X_2*W_2 + ... + X_n*W_n / sqrt(W_1**2 + W_2**2 + ... W_n**2)
             = weighted sum of X and normalized W
           here data = self.query_emb, weight = query_movie_mask '''
        self.query_emb = tf.nn.embedding_lookup(self.w_anime_id, self.anime_id)
        query_movie_mask = tf.expand_dims(tf.nn.l2_normalize(tf.to_float(tf.sequence_mask(self.anime_id)), 1), -1)
        self.query_emb = tf.reduce_sum(self.query_emb * query_movie_mask, 1)
        self.query_bias = tf.matmul(self.query_emb, self.b_anime_id[:, tf.newaxis])

        # candidate_movie embedding
        self.candidate_emb = tf.nn.embedding_lookup(self.w_candidate_movie_id, self.candidate_movie_id)

        # genres embedding
        '''sqrtn aggregation(pooling), X: data, W: weight
               X_1*W_1 + X_2*W_2 + ... + X_n*W_n / sqrt(W_1**2 + W_2**2 + ... W_n**2)
             = weighted sum of X and normalized W
           here data = self.genres_emb, weight = genres_mask '''
        self.genre_emb = tf.nn.embedding_lookup(self.w_genre, tf.to_int32(self.genre))
        genre_mask = tf.expand_dims( tf.nn.l2_normalize(tf.to_float(tf.sequence_mask(self.genre_len)), 1), -1)
        self.genre_emb = tf.reduce_sum(self.genre_emb * genre_mask, 1)

In [20]:
def graph_dnn(self):
    with tf.variable_scope("dnn"):
        init_fn = tf.glorot_uniform_initializer()
        # encode [item embedding + item metadata]
        self.item_repr = tf.concat([self.candidate_emb, self.genre_emb, self.rating[:, tf.newaxis], self.member[:, tf.newaxis]], 1)
        self.candidate_bias = tf.matmul(self.item_repr, self.b_candidate_movie_id[:, tf.newaxis])
        
        # Do: 目前兩層DNN, 可以試著增加或減少hidden layer, 但是最後要跟query_emb內積, 最後一層維度必須 = dim
        # Example:
            # self.item_repr = tf.layers.dense(self.item_repr, 64, kernel_initializer=init_fn, activation=tf.nn.relu)
            # self.item_repr = tf.layers.dense(self.item_repr, 32, kernel_initializer=init_fn, activation=tf.nn.relu)
            # self.item_repr = tf.layers.dense(self.item_repr, self.dim, kernel_initializer=init_fn, activation=tf.nn.relu)

        # Do: 若有overfitting後可嘗試dropout
        # dp_scale = 0.5
        self.item_repr = tf.layers.dense(self.item_repr, self.dim, kernel_initializer=init_fn, activation=tf.nn.relu)
        # self.item_repr = tf.layers.dropout(self.item_repr, dp_scale, training=self.isTrain)
        self.item_repr = tf.layers.dense(self.item_repr, self.dim, kernel_initializer=init_fn, activation=tf.nn.relu)
        # self.item_repr = tf.layers.dropout(self.item_repr, dp_scale, training=self.isTrain)

In [21]:
def graph_computation(self):
    with tf.variable_scope("computation"):
        infer = tf.reduce_sum(self.query_emb * self.item_repr, 1, keep_dims=True)
        infer = tf.add(infer, self.b_global)
        infer = tf.add(infer, self.query_bias)
        self.infer = tf.add(infer, self.candidate_bias, name="infer")

        # one query for all items
        self.pred = tf.matmul(self.query_emb, tf.transpose(self.item_repr)) + \
                    tf.reshape(self.candidate_bias, (1, -1)) + self.query_bias + self.b_global

In [22]:
def graph_loss(self):
    with tf.variable_scope("loss"):
        self.loss = tf.losses.mean_squared_error(labels=self.rating_x[:, tf.newaxis], predictions=self.infer)
        # for eval
        self.rmse_loss = tf.sqrt(self.loss)
        self.mae_loss = tf.reduce_mean(tf.abs(self.infer - self.rating_x[:, tf.newaxis]))

    with tf.variable_scope("train"):
        # Do: 嘗試不同的Optimizer
        #self.train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss)
        # self.train_op = tf.train.MomentumOptimizer(learning_rate, 0.9, use_nesterov=True).minimize(self.loss)
        # self.train_op = tf.train.AdagradOptimizer(learning_rate).minimize(self.loss)
        # self.train_op = tf.train.RMSPropOptimizer(learning_rate).minimize(self.loss)
         self.train_op = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

In [23]:
def fit(self, sess, trainGen, testGen, reset=False, nEpoch=50):
    sess.run(tf.global_variables_initializer())
    if reset:
        print("reset model: clean model dir: {} ...".format(self.modelDir))
        self.resetModel(self.modelDir)
    # try: 試著重上次儲存的model再次training
    self.ckpt(sess, self.modelDir)

    start = time.time()
    print("%s\t%s\t%s\t%s" % ("Epoch", "Train Error", "Val Error", "Elapsed Time"))
    minLoss = 1e7
    for ep in range(1, nEpoch + 1):
        tr_loss, tr_total = 0, 0
        for i, data in enumerate(trainGen(), 1):
            loss, _ = sess.run([self.rmse_loss, self.train_op], feed_dict=self.feed_dict(data, mode="train"))
            tr_loss += loss ** 2 * len(data["anime_id"])
            tr_total += len(data["anime_id"])
            print("\rtrain loss: {:.3f}".format(loss), end="")
        if testGen is not None:
            epochLoss = self.epochLoss(sess, testGen)

        tpl = "\r%02d\t%.3f\t\t%.3f\t\t%.3f secs"
        if minLoss > epochLoss:
            tpl += ", saving ..."
            self.saver.save(sess, os.path.join(self.modelDir, 'model'), global_step=ep)
            minLoss = epochLoss

        end = time.time()
        print(tpl % (ep, np.sqrt(tr_loss / tr_total), epochLoss, end - start))
        start = end
    return self

In [25]:
nUsers, nMovies = len(mergedata['user_id'].unique()), len(mergedata['anime_id'].unique())


In [42]:
# hyper parameters
n_batch = 128
# Do: 嘗試不同的learning_rate [0.1, 0.001, 0.0001]
learning_rate = 0.001
# Do: 嘗試不同的dim [8, 16, 20, 32]
dim = 5
# 非必要: 改動model dir
modelDir = "./model"
    
tf.reset_default_graph()
model = ModelMfDNN(
            n_items=nMovies,
            n_genre=n_genres,
            dim=dim,
            learning_rate=learning_rate,
            modelDir=modelDir)

ValueError: Shape must be rank 2 but is rank 3 for 'embedding/MatMul' (op: 'MatMul') with input shapes: [?,?,5], [5,1].